In [ ]:
import random

# Define card values
card_values = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'J': 11, 'Q': 12, 'K': 13, 'A': 14}

# Function to create a deck of cards for each player
def create_player_deck(suit):
    ranks = ['2', '3', '4', '5', '6', '7', '8', '9', 'J', 'Q', 'K', 'A']
    player_deck = [(rank, suit) for rank in ranks]
    return player_deck

# Function to calculate points of a card
def calculate_points(card):
    return card_values[card[0]]

# Function to determine the winner of the bid
def determine_winner(bids):
    max_bid = max(bids)
    if bids.count(max_bid) == 1:
        return bids.index(max_bid)
    else:
        max_bidders = [i for i, bid in enumerate(bids) if bid == max_bid]
        return max_bidders


# Function for code-controlled player to make bids based on the enhanced strategy
def code_make_bid(player_deck, player_scores, current_diamond_card, round_number, previous_bid=None):
    # Evaluate the current score and position of Player 1
    player1_score = player_scores[0]
    player1_position = sum(score > player1_score for score in player_scores[1:])

    # Calculate the threshold score for aggressive bidding
    aggressive_threshold = 20  # Adjust as needed based on game dynamics

    # Evaluate the value of the current bid based on the diamond card being auctioned
    diamond_values = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'J': 11, 'Q': 12, 'K': 13, 'A': 14}
    current_diamond_value = diamond_values[current_diamond_card[0]]

    # Exclude the previously bid card from the available options
    available_cards = [card for card in player_deck if card != previous_bid]

    # Count the number of high-value cards remaining in Player 1's deck
    high_value_cards = sum(1 for card in available_cards if diamond_values[card[0]] >= 10)

    # Determine the bidding strategy based on the current state of the game
    if not available_cards:
        # No available cards to bid
        return None

    if round_number < 5:
        # Early rounds: Bid aggressively on high-value cards
        high_value_available_cards = [card for card in available_cards if card[0] in ['K', 'A']]
        if high_value_available_cards:
            bid_card = random.choice(high_value_available_cards)
        else:
            bid_card = random.choice(available_cards)
    elif round_number >= 5 and round_number < 10:
        # Middle rounds: Adjust bidding strategy based on score distribution and remaining high-value cards
        if player1_position == 0:
            # Player 1 is leading or tied for the lead
            if high_value_cards > 2:
                # Many high-value cards remaining: Bid aggressively
                high_value_available_cards = [card for card in available_cards if card[0] in ['K', 'A']]
                if high_value_available_cards:
                    bid_card = random.choice(high_value_available_cards)
                else:
                    bid_card = random.choice(available_cards)
            else:
                # Few high-value cards remaining: Bid conservatively
                bid_card = random.choice(available_cards)
        elif player1_position > 0:
            # Player 1 is trailing
            if player1_score < aggressive_threshold:
                # Below aggressive bidding threshold: Bid aggressively
                high_value_available_cards = [card for card in available_cards if card[0] in ['K', 'A']]
                if high_value_available_cards:
                    bid_card = random.choice(high_value_available_cards)
                else:
                    bid_card = random.choice(available_cards)
            else:
                # Above aggressive bidding threshold: Bid conservatively
                bid_card = random.choice(available_cards)
    else:
        # Late rounds: Adjust bidding strategy based on remaining high-value cards
        if high_value_cards > 2:
            # Many high-value cards remaining: Bid aggressively
            high_value_available_cards = [card for card in available_cards if card[0] in ['K', 'A']]
            if high_value_available_cards:
                bid_card = random.choice(high_value_available_cards)
            else:
                bid_card = random.choice(available_cards)
        else:
            # Few high-value cards remaining: Bid conservatively
            bid_card = random.choice(available_cards)

    # Remove the chosen card from the player's deck
    player_deck.remove(bid_card)
    return bid_card


# Function for user-controlled player to make a bid
def user_make_bid(player_deck):
    print("You are Player 3.")
    print("Your remaining cards:", player_deck)
    user_input = input("Enter your bid (choose a card from your remaining cards): ").upper()
    while user_input not in [card[0] for card in player_deck]:
        print("Invalid input. Please choose a card from your remaining cards.")
        user_input = input("Enter your bid: ").upper()
    card = [card for card in player_deck if card[0] == user_input][0]
    player_deck.remove(card)
    return card

# Function to distribute points among winners
def distribute_points(player_scores, winner_indices, points):
    points_per_winner = points // len(winner_indices)
    for idx in winner_indices:
        player_scores[idx] += points_per_winner

# Function to play the game
def play_diamonds():
    # Initialize player scores
    player_scores = [0, 0, 0]

    # Create deck for each player and remove diamond cards
    player1_deck = create_player_deck('H')
    player2_deck = create_player_deck('S')
    user_deck = create_player_deck('C')

    # Diamond cards
    diamond_cards = [('2', 'D'), ('3', 'D'), ('4', 'D'), ('5', 'D'), ('6', 'D'), ('7', 'D'), ('8', 'D'), ('9', 'D'), ('J', 'D'), ('Q', 'D'), ('K', 'D'), ('A', 'D')]

    # Shuffle diamond cards
    random.shuffle(diamond_cards)

    # Play the game
    for round_number, diamond_card in enumerate(diamond_cards, start=1):
        bids = []

        print("\nDiamond card:", diamond_card[0])

        # Player 1 (code-controlled)
        player1_card = code_make_bid(player1_deck, player_scores, diamond_card, round_number)
        bids.append(calculate_points(player1_card[0]))

        # Player 2 (code-controlled)
        player2_card = code_make_bid(player2_deck, player_scores, diamond_card, round_number)
        bids.append(calculate_points(player2_card[0]))

        # Player 3 (user-controlled)
        user_card = user_make_bid(user_deck)
        print("Your bid:", user_card[0])
        bids.append(calculate_points(user_card[0]))

        # Print player bids after user input
        print("Player 1 bid:", player1_card[0])
        print("Player 2 bid:", player2_card[0])

        # Determine the winner(s) of the bid
        winner_indices = determine_winner(bids)

        # Distribute points to the winner(s)
        if isinstance(winner_indices, int):
            points = calculate_points(diamond_card[0])
            player_scores[winner_indices] += points
            print(f"Player {winner_indices + 1} wins the bid and gets {points} points.")
        else:
            points = calculate_points(diamond_card[0])
            points_per_winner = points // len(winner_indices)  # Calculate points per winner
            for idx in winner_indices:
                player_scores[idx] += points_per_winner
                print(f"Player {idx + 1} ties for the bid and gets {points_per_winner} points.")

        print("Player Scores:", player_scores)

# Play the game
play_diamonds()



Diamond card: 8
You are Player 3.
Your remaining cards: [('2', 'C'), ('3', 'C'), ('4', 'C'), ('5', 'C'), ('6', 'C'), ('7', 'C'), ('8', 'C'), ('9', 'C'), ('J', 'C'), ('Q', 'C'), ('K', 'C'), ('A', 'C')]
Enter your bid (choose a card from your remaining cards): Q
Your bid: Q
Player 1 bid: K
Player 2 bid: K
Player 1 ties for the bid and gets 4 points.
Player 2 ties for the bid and gets 4 points.
Player Scores: [4, 4, 0]

Diamond card: A
You are Player 3.
Your remaining cards: [('2', 'C'), ('3', 'C'), ('4', 'C'), ('5', 'C'), ('6', 'C'), ('7', 'C'), ('8', 'C'), ('9', 'C'), ('J', 'C'), ('K', 'C'), ('A', 'C')]
Enter your bid (choose a card from your remaining cards): A
Your bid: A
Player 1 bid: A
Player 2 bid: A
Player 1 ties for the bid and gets 4 points.
Player 2 ties for the bid and gets 4 points.
Player 3 ties for the bid and gets 4 points.
Player Scores: [8, 8, 4]

Diamond card: Q
You are Player 3.
Your remaining cards: [('2', 'C'), ('3', 'C'), ('4', 'C'), ('5', 'C'), ('6', 'C'), ('7', '